# Testing Conversion
This notebook is used for some simple poking around and testing the components 
needed for converting a networkx graph with standard ONNX operator types to a 
networkx graph with tensorflow operator types.

The basic conversion flow is:
networkx graph (onnx ops) -> standard onnx -> tf -> networkx graph (tf ops)

In [2]:
import os
import pathlib
import json

import numpy as np
import tensorflow as tf

import onnx
from onnx.shape_inference import infer_shapes

from youngernns.data.network import Network
from youngernns.nn_translation import networkx_to_onnx

## Global Variables

In [3]:
DATA_PATH = os.path.join("..","data","filter_series_with_attributes_paper")
OUTPUT_DIR = os.path.join("..", "output")

## Loading Data

In [4]:
def get_graph_paths():
    # find all paths for network folders DATA_PATH/gaphd_id/network/
    graph_folders = []
    for root, dirs, files in os.walk(DATA_PATH):
        if 'network' in dirs:
            graph_folders.append(os.path.join(root, 'network'))
    return graph_folders

In [5]:
graph_paths = get_graph_paths()
print(f"Found {len(graph_paths)} graphs")
print(f"Example of first graph path: {graph_paths[0]}")

Found 7629 graphs
Example of first graph path: ../data/filter_series_with_attributes_paper/0ad2b9b313487b294428174cf56d7d22/network


Let's test the conversion on the first graph path

In [6]:

NN = Network()
NN.load(pathlib.Path(graph_paths[0]))

onnx_model = networkx_to_onnx(NN.graph, input_shape=(1, 128, 128, 3), output_shape=(1,2))
shaped_onnx_model = infer_shapes(onnx_model)

In [7]:
shaped_onnx_model.graph.node

[input: "data"
output: "node_0"
name: "node_0"
op_type: "Transpose"
attribute {
  name: "perm"
  ints: 0
  ints: 3
  ints: 1
  ints: 2
  type: INTS
}
, input: "node_0"
input: "Conv_weights_1"
output: "node_1"
name: "node_1"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "group"
  i: 1
  type: INT
}
attribute {
  name: "kernel_shape"
  ints: 3
  ints: 3
  type: INTS
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}
, input: "node_1"
output: "node_2"
name: "node_2"
op_type: "Relu"
, input: "node_2"
output: "node_3"
name: "node_3"
op_type: "MaxPool"
attribute {
  name: "ceil_mode"
  i: 0
  type: INT
}
attribute {
  name: "kernel_shape"
  ints: 2
  ints: 2
  type: INTS
}
attribute {
  name: "storage_order"
  i: 0
  type: INT
}
attribute {
  name: "strides"
  ints: 2
  ints: 2
  type: INTS
}
, input: "node_3"
input: "Conv_weights_4"
output: "node_4"
name: "node_4"
op_type: "Conv"
attribute {
  name: "dilations"
  ints:

In [8]:
# Load tensorflow tensor image
image_path = os.path.join("..", "output", "tf_images", "corgi_128_tensor.npy")

# Load tensor
def load_npy_tensor(file_path):
    # Load the .npy file
    np_tensor = np.load(file_path)
    
    # Convert the numpy array to a TensorFlow tensor
    tf_tensor = tf.convert_to_tensor(np_tensor, dtype=tf.float32)
    return tf_tensor

img = load_npy_tensor(image_path)

In [9]:
img.shape

TensorShape([1, 128, 128, 3])

In [10]:
from onnx.reference import ReferenceEvaluator
oinf = ReferenceEvaluator(shaped_onnx_model, verbose=3)
oinf.run(None, {"data": img})

 +C Conv_weights_1: float32:(3, 3, 3, 3) in [0.018789799883961678, 0.9883738160133362]
 +C Conv_weights_4: float32:(3, 3, 3, 3) in [0.004695476032793522, 0.9988470077514648]
 +C Conv_weights_7: float32:(3, 3, 3, 3) in [0.011714084073901176, 0.990338921546936]
 +C Reshape_shape_11: int64:(2,) in [-1, 1]
 +C Matmul_Y_12: float32:(588, 2) in [0.0005459649255499244, 0.9998085498809814]
 +C Add_Y_13: float32:(1, 2) in [0.049499742686748505, 0.6854671239852905]
 +C Matmul_Y_15: float32:(2, 2) in [0.10185461491346359, 0.3165411353111267]
 +C Add_Y_16: float32:(1, 2) in [0.2550637722015381, 0.7505366802215576]
 +I data: tf.Tensor(
[[[[0.34509805 0.30980393 0.27450982]
   [0.34901962 0.31764707 0.28627452]
   [0.34509805 0.31764707 0.28627452]
   ...
   [0.16078432 0.15686275 0.15686275]
   [0.16470589 0.16862746 0.16078432]
   [0.1764706  0.1764706  0.16862746]]

  [[0.3529412  0.31764707 0.28627452]
   [0.34901962 0.3254902  0.29411766]
   [0.33333334 0.3137255  0.28235295]
   ...
   [0.16078

[array([[0., 1.]], dtype=float32)]

In [11]:
# Save the onnx model
onnx_path = os.path.join(OUTPUT_DIR, "onnx_model.onnx")
with open(onnx_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

In [12]:
onnx.checker.check_model(onnx_path)

Now let's try and convert the onnx model to tensorflow format.

In [13]:
! onnx2tf -i ../output/onnx_model.onnx -o ../output/tf_onnx_model -b 1 -kat data


Model optimizing started ============================================================
Traceback (most recent call last):
  File "/home/wdm/YoungerNNs/.venv/lib/python3.12/site-packages/onnx2tf/onnx2tf.py", line 645, in convert
    result = subprocess.check_output(
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/subprocess.py", line 466, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/subprocess.py", line 571, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['onnxsim', '../output/onnx_model.onnx', '../output/onnx_model.onnx']' returned non-zero exit status 1.


Automatic generation of each OP name started ========================================
Automatic generation of each OP name complete!

Model loaded ========================================================================

Model co

Now, we load the tensorflow model and look at the operator names.

In [14]:
# Load tensorflow model from output/tf_onnx_model
tf_model = tf.saved_model.load("../output/tf_onnx_model")

In [15]:
type(tf_model)

tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject

Now let's look at the operators in the tensorflow graph

In [16]:
concrete_func = tf_model.signatures['serving_default']
tf_graph =concrete_func.graph.as_graph_def()

In [17]:
tf_graph

node {
  name: "data"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "data"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 1
        }
        dim {
          size: 128
        }
        dim {
          size: 128
        }
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "unknown"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 3
        }
        dim {
          size: 3
        }
        dim {
          size: 3
        }
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "unknown_0"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 3
        }
  

In [18]:
function_library = tf_graph.library
function_defs = [f for f in function_library.function]

In [19]:
node_def_attrs = [{'op': n.op, 'input': n.input, "output": n.name} for n in function_defs[0].node_def]
node_def_attrs

[{'op': 'Const',
  'input': [],
  'output': 'model_1/tf.compat.v1.transpose/transpose/perm'},
 {'op': 'Transpose',
  'input': ['data', 'model_1/tf.compat.v1.transpose/transpose/perm:output:0'],
  'output': 'model_1/tf.compat.v1.transpose/transpose'},
 {'op': 'Const',
  'input': [],
  'output': 'model_1/tf.compat.v1.transpose_1/transpose/perm'},
 {'op': 'Transpose',
  'input': ['model_1/tf.compat.v1.transpose/transpose:y:0', 'model_1/tf.compat.v1.transpose_1/transpose/perm:output:0'],
  'output': 'model_1/tf.compat.v1.transpose_1/transpose'},
 {'op': 'Conv2D',
  'input': ['model_1/tf.compat.v1.transpose_1/transpose:y:0', 'model_1_tf_nn_convolution_convolution_filter'],
  'output': 'model_1/tf.nn.convolution/convolution'},
 {'op': 'Relu',
  'input': ['model_1/tf.nn.convolution/convolution:output:0'],
  'output': 'model_1/tf.nn.relu/Relu'},
 {'op': 'MaxPool',
  'input': ['model_1/tf.nn.relu/Relu:activations:0'],
  'output': 'model_1/tf.nn.max_pool2d/MaxPool2d'},
 {'op': 'Conv2D',
  'input

In [20]:
with open(os.path.join("..", "config", "tflm.json"), "r") as f:
    tflm_data = json.loads(f.read())

valid_node_defs = [n for n in node_def_attrs if n['op'] in tflm_data['operators'] and n['input'] != []]

In [21]:
node_shapes = [n.attr.get('shape', None) for n in tf_graph.node]
node_shapes

[shape {
   dim {
     size: 1
   }
   dim {
     size: 128
   }
   dim {
     size: 128
   }
   dim {
     size: 3
   }
 },
 shape {
   dim {
     size: 3
   }
   dim {
     size: 3
   }
   dim {
     size: 3
   }
   dim {
     size: 3
   }
 },
 shape {
   dim {
     size: 3
   }
   dim {
     size: 3
   }
   dim {
     size: 3
   }
   dim {
     size: 3
   }
 },
 shape {
   dim {
     size: 3
   }
   dim {
     size: 3
   }
   dim {
     size: 3
   }
   dim {
     size: 3
   }
 },
 shape {
   dim {
     size: 588
   }
   dim {
     size: 2
   }
 },
 shape {
   dim {
     size: 1
   }
   dim {
     size: 2
   }
 },
 shape {
   dim {
     size: 2
   }
   dim {
     size: 2
   }
 },
 shape {
   dim {
     size: 1
   }
   dim {
     size: 2
   }
 },
 None,
 None]

In [22]:
valid_node_defs

[{'op': 'Transpose',
  'input': ['data', 'model_1/tf.compat.v1.transpose/transpose/perm:output:0'],
  'output': 'model_1/tf.compat.v1.transpose/transpose'},
 {'op': 'Transpose',
  'input': ['model_1/tf.compat.v1.transpose/transpose:y:0', 'model_1/tf.compat.v1.transpose_1/transpose/perm:output:0'],
  'output': 'model_1/tf.compat.v1.transpose_1/transpose'},
 {'op': 'Conv2D',
  'input': ['model_1/tf.compat.v1.transpose_1/transpose:y:0', 'model_1_tf_nn_convolution_convolution_filter'],
  'output': 'model_1/tf.nn.convolution/convolution'},
 {'op': 'Relu',
  'input': ['model_1/tf.nn.convolution/convolution:output:0'],
  'output': 'model_1/tf.nn.relu/Relu'},
 {'op': 'Conv2D',
  'input': ['model_1/tf.nn.max_pool2d/MaxPool2d:output:0', 'model_1_tf_nn_convolution_1_convolution_filter'],
  'output': 'model_1/tf.nn.convolution_1/convolution'},
 {'op': 'Relu',
  'input': ['model_1/tf.nn.convolution_1/convolution:output:0'],
  'output': 'model_1/tf.nn.relu_1/Relu'},
 {'op': 'Conv2D',
  'input': ['mo

Now, let's try and see if we can get the op, attribute, and shape information into 
networkx graph nodes and use the input/output information to create edges in the networkx graph. 